# Streaming
# 스트리밍

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

스트리밍은 데이터 생성과 사용자가 받는 시점 사이의 지연 시간을 줄여줍니다.
에이전트에서 자주 사용되는 두 가지 타입이 있습니다:

## Setup
## 설정

Load and/or check for needed environmental variables

필요한 환경 변수를 로드하거나 확인합니다

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

In [ ]:
from langchain.agents import create_agent

In [ ]:
agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)
## 스트리밍 없이 (invoke)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

## values
## values 모드
You have seen this streaming mode in our examples so far.

지금까지 예제에서 이 스트리밍 모드를 보셨습니다.

step이 끝나면 메세지 전송 (ex. tool 호출 응답 다음 등)

In [ ]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## messages
## messages 모드
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

messages 모드는 토큰 단위로 데이터를 스트리밍합니다 - 가능한 가장 낮은 지연 시간입니다. 챗봇과 같은 인터랙티브 애플리케이션에 적합합니다.

In [ ]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

## Tools can stream too!
## 도구도 스트리밍할 수 있습니다!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

스트리밍은 일반적으로 최종 결과가 준비되기 전에 사용자에게 정보를 전달하는 것을 의미합니다. 이것이 유용한 경우가 많습니다. `get_stream_writer`를 사용하면 직접 만든 소스에서 `custom` 데이터를 쉽게 스트리밍할 수 있습니다.

In [ ]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

## Try different modes on your own!
## 직접 다른 모드를 시도해보세요!
Modify the stream mode and the select to produce different results.

스트림 모드와 선택 조건을 수정하여 다른 결과를 만들어 보세요.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])